## 환경 설정 및 라이브러리 불러오기

In [ ]:
# konlpy 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
!pip install attrdict
!pip install -q -U transformers
!pip install soynlp
!pip install emoji

In [ ]:
# from pororo import Pororo # 카카오 포로로
import pandas as pd
import os
import json
import numpy as np
import shutil
# from hanspell import spell_checker  # 한스펠

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from datetime import datetime, timezone, timedelta
import random
from tqdm import tqdm

from attrdict import AttrDict
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils import *
from torch.utils.data import DataLoader
from torch.optim import Adam, AdamW

import re
import emoji
from soynlp.normalizer import repeat_normalize

from transformers import logging, get_linear_schedule_with_warmup

from transformers import ( 
    BertConfig,
    ElectraConfig
)
from transformers import (
    BertModel,
    AutoModel, 
    ElectraForSequenceClassification,
    BertForSequenceClassification,
    AlbertForSequenceClassification
)
from transformers import AutoModel, AutoTokenizer

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from konlpy.tag import Okt, Mecab

okt = Okt()
mecab = Mecab()

In [ ]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(2022)

In [ ]:
# 사용할 GPU 지정
print("number of GPUs: ", torch.cuda.device_count())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
use_cuda = torch.cuda.is_available()
print("Does GPU exist? : ", use_cuda)
DEVICE = torch.device("cuda" if use_cuda else "cpu")

number of GPUs:  1
Does GPU exist? :  True


In [ ]:
# True 일 때 코드를 실행하면 example 등을 보여줌
DEBUG = True

In [ ]:
# 경로
DATA_PATH = '/content/drive/MyDrive/AI_team/competition3'

## 데이터 확인

In [ ]:
# team best score submit
team_best_submit = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/submission/team_best_score_7767.csv')

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/data/train.csv', encoding = 'UTF-8-SIG')
test_df = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/data/test.csv', encoding = 'UTF-8-SIG')
train_df.head()

,title,comment,bias,hate
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none


In [ ]:
train_df[train_df['bias']=='none'].shape[0]

5490

In [ ]:
train_df[train_df['bias']=='gender'].shape[0]

1299

In [ ]:
train_df[train_df['bias']=='others'].shape[0]

1578

In [ ]:
team_best_submit[team_best_submit['bias']=='none'].shape[0]

305

In [ ]:
team_best_submit[team_best_submit['bias']=='gender'].shape[0]

105

In [ ]:
team_best_submit[team_best_submit['bias']=='others'].shape[0]

101

## Test 데이터 전처리, 최대한 Train과 비슷하게

In [ ]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
punct = "/-'.,#$%\'()*+-!/:·;→<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' + f'{emojis}' # !, ? 넣을지 말지 고민중
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", 'ᆢ':'', "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', } 
specials = {'\u200b': ' ', '…': '', '\ufeff': '', 
            '넘도':' 놈도 ', 'OOO' : ' ㅅㅂ ', 'ㄹㅇ':'정말', '❤️':'♥', '♡':'♥',
            '짜식':'자식', '손가릭':'손가락','틀 딱':'틀딱','아 줌 만':'아줌마','연병':'염병'
            }    # '♥': ' 하트 ', '★': ' 스타 '
def clean_punc(text, punct, mapping): 
    for p in mapping: 
        text = text.replace(p, mapping[p]) 
    for p in punct:
        text = text.replace(p, ' ')
    for s in specials:
        text = text.replace(s, specials[s])
    normalized_text = repeat_normalize(text, num_repeats=2)
    if len(str(normalized_text)) < 6:
        text = (normalized_text + ' ') * 2
    elif 6 <= len(str(normalized_text)) < 9:
        text = (normalized_text + ' ') * 2
    else:
        text = normalized_text
    return text

In [ ]:
from konlpy.tag import Okt, Mecab

okt = Okt()
mecab = Mecab()

In [ ]:
test_df['clean_comment'] = test_df['comment'].apply(lambda x: clean_punc(x, punct, punct_mapping))

In [ ]:
test_df.drop(['title'], axis=1, inplace=True)

In [ ]:
# TOKENIZER = AutoTokenizer.from_pretrained("beomi/beep-KcELECTRA-base-hate")  # hate
TOKENIZER = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")  # bias

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
test_df['token'] = test_df['clean_comment'].apply(lambda x: TOKENIZER.tokenize(x))
test_df['bias'] = team_best_submit['bias']
test_df.drop(['comment'], axis=1, inplace=True)
test_df

,ID,clean_comment,token,bias
0,0,둘다 넘 좋다 행복하세요,"[둘다, 넘, 좋다, 행복, ##하세요]",none
1,1,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,"[근데, 만원, ##이하, ##는, 현금, ##결제, ##만, 하라고, 써, ##놓...",none
2,2,누군데 얘네? 누군데 얘네?,"[누군데, 얘네, ?, 누군데, 얘네, ?]",none
3,3,쑈 하지마라 자식아 음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 기어...,"[쑈, 하지마라, 자식, ##아, 음주, 1번, ##은, 실수, 2번, ##은, 고...",none
4,4,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,"[안, ##검, ##하, ##수, 가지고, 있는, 분, ##께, 희망을, 주고, 싶...",none
...,...,...,...,...
506,506,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,"[얜, 그냥, 봐도, 아니다, ㅋ, 고소, ##당하면, 어마어마한, 금액, 물어, ...",none
507,507,대박 게스트 꼭 봐야징 컨셉이 바뀌니깐 재미지넹,"[대박, 게스트, 꼭, 봐야, ##징, 컨셉, ##이, 바뀌, ##니깐, 재미, #...",none
508,508,성형으로 다 뜯어고쳐놓고 예쁜척 성형 전 니 얼굴 다 알고있다 순자처럼 된장냄새나게...,"[성형, ##으로, 다, 뜯어, ##고, ##쳐, ##놓고, 예쁜, ##척, 성형,...",others
509,509,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,"[분위기, ##는, 비슷, ##하다, ##만, 전혀, ##다른, 전개, ##던데, ...",others


In [ ]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))

In [ ]:
test_df['okt'] = test_df['clean_comment'].apply(lambda x: okt.morphs(x))

In [ ]:
test_df['okt_clean'] = test_df['okt'].apply(lambda x:listToString(x))

In [ ]:
test_df['mecab'] = test_df['clean_comment'].apply(lambda x: mecab.morphs(x))
test_df['mecab_clean'] = test_df['mecab'].apply(lambda x:listToString(x))

In [ ]:
test_df.drop(['okt'], axis=1, inplace=True)

In [ ]:
test_df['clean_comment2'] = test_df['okt_clean'].apply(lambda x: clean_punc(x, punct, punct_mapping))

In [ ]:
test_df

,ID,comment,clean_comment,okt_clean,mecab,mecab_clean,clean_comment2
0,0,둘다 넘 좋다~행복하세요,둘다 넘 좋다 행복하세요,둘다 넘 좋다 행복하세요,"[둘, 다, 넘, 좋, 다, 행복, 하, 세요]",둘 다 넘 좋 다 행복 하 세요,둘다 넘 좋다 행복하세요
1,1,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만 원 이하 는 현금 결제 만 하라 고 써놓은 집 우리나라 에 엄청 많은데,"[근데, 만, 원, 이, 하, 는, 현금, 결제, 만, 하, 라고, 써, 놓, 은,...",근데 만 원 이 하 는 현금 결제 만 하 라고 써 놓 은 집 우리나라 에 엄청 많 은데,근데 만 원 이하 는 현금 결제 만 하라 고 써놓은 집 우리나라 에 엄청 많은데
2,2,누군데 얘네?,누군데 얘네? 누군데 얘네?,누군데 얘 네 ? 누군데 얘 네 ?,"[누군데, 얘, 네, ?, 누군데, 얘, 네, ?]",누군데 얘 네 ? 누군데 얘 네 ?,누군데 얘 네 ? 누군데 얘 네 ?
3,3,"쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...",쑈 하지마라 자식아 음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 기어...,쑈 하지마라 자식 아 음주 1 번은 실수 2 번은 고의 3 번은 인간 쓰레기 다 슬...,"[쑈, 하, 지, 마, 라, 자식, 아, 음주, 1, 번, 은, 실수, 2, 번, ...",쑈 하 지 마 라 자식 아 음주 1 번 은 실수 2 번 은 고의 3 번 은 인간 쓰...,쑈 하지마라 자식 아 음주 1 번은 실수 2 번은 고의 3 번은 인간 쓰레기 다 슬...
4,4,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검 하수 가지 고 있는 분 께 희망 을 주고 싶은건 가요 ? 수술 하면 이렇게 자...,"[안검, 하, 수, 가지, 고, 있, 는, 분, 께, 희망, 을, 주, 고, 싶, ...",안검 하 수 가지 고 있 는 분 께 희망 을 주 고 싶 은 건가요 ? 수술 하 면 ...,안검 하수 가지 고 있는 분 께 희망 을 주고 싶은건 가요 ? 수술 하면 이렇게 자...
...,...,...,...,...,...,...,...
506,506,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소 당하면 어마어마한 금액 물어줘 야할 껄 ?,"[얜, 그냥, 봐도, 아니, 다, ㅋ, 고소, 당하, 면, 어마어마, 한, 금액, ...",얜 그냥 봐도 아니 다 ㅋ 고소 당하 면 어마어마 한 금액 물 어 줘야 할껄 ?,얜 그냥 봐도 아니다 ㅋ 고소 당하면 어마어마한 금액 물어줘 야할 껄 ?
507,507,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야징 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야 징 컨셉 이 바뀌니깐 재미 지 넹,"[대박, 게스트, 꼭, 봐, 야징, 컨셉, 이, 바뀌, 니깐, 재미, 지, 넹]",대박 게스트 꼭 봐 야징 컨셉 이 바뀌 니깐 재미 지 넹,대박 게스트 꼭 봐야 징 컨셉 이 바뀌니깐 재미 지 넹
508,508,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,성형으로 다 뜯어고쳐놓고 예쁜척 성형 전 니 얼굴 다 알고있다 순자처럼 된장냄새나게...,성형 으로 다 뜯어 고쳐놓고 예쁜 척 성형 전 니 얼굴 다 알고있다 순자 처럼 된장...,"[성형, 으로, 다, 뜯어고쳐, 놓, 고, 예쁜, 척, 성형, 전, 니, 얼굴, 다...",성형 으로 다 뜯어고쳐 놓 고 예쁜 척 성형 전 니 얼굴 다 알 고 있 다 순자 처...,성형 으로 다 뜯어 고쳐놓고 예쁜 척 성형 전 니 얼굴 다 알고있다 순자 처럼 된장...
509,509,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기 는 비슷하다만 전혀 다른 전개 던데 무슨 ㅋㅋㄱ 우리나라 사람 들 은 분위기...,"[분위기, 는, 비슷, 하, 다만, 전혀, 다른, 전개, 던데, 무슨, ㅋㅋ, ㄱ,...",분위기 는 비슷 하 다만 전혀 다른 전개 던데 무슨 ㅋㅋ ㄱ 우리나라 사람 들 은 ...,분위기 는 비슷하다만 전혀 다른 전개 던데 무슨 ㅋㅋㄱ 우리나라 사람 들 은 분위기...


In [ ]:
# TOKENIZER = AutoTokenizer.from_pretrained("beomi/beep-KcELECTRA-base-hate")  # hate
TOKENIZER = AutoTokenizer.from_pretrained("beomi/beep-KcELECTRA-base-bias")  # bias


Downloading:   0%|          | 0.00/544 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/769k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
test_df['token'] = test_df['clean_comment'].apply(lambda x: TOKENIZER.tokenize(x))
test_df

,ID,comment,clean_comment,okt_clean,mecab,mecab_clean,clean_comment2,token
0,0,둘다 넘 좋다~행복하세요,둘다 넘 좋다 행복하세요,둘다 넘 좋다 행복하세요,"[둘, 다, 넘, 좋, 다, 행복, 하, 세요]",둘 다 넘 좋 다 행복 하 세요,둘다 넘 좋다 행복하세요,"[둘다, 넘, 좋다, 행복, ##하세요]"
1,1,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만 원 이하 는 현금 결제 만 하라 고 써놓은 집 우리나라 에 엄청 많은데,"[근데, 만, 원, 이, 하, 는, 현금, 결제, 만, 하, 라고, 써, 놓, 은,...",근데 만 원 이 하 는 현금 결제 만 하 라고 써 놓 은 집 우리나라 에 엄청 많 은데,근데 만 원 이하 는 현금 결제 만 하라 고 써놓은 집 우리나라 에 엄청 많은데,"[근데, 만원, ##이하, ##는, 현금, ##결제, ##만, 하라고, 써, ##놓..."
2,2,누군데 얘네?,누군데 얘네? 누군데 얘네?,누군데 얘 네 ? 누군데 얘 네 ?,"[누군데, 얘, 네, ?, 누군데, 얘, 네, ?]",누군데 얘 네 ? 누군데 얘 네 ?,누군데 얘 네 ? 누군데 얘 네 ?,"[누군데, 얘네, ?, 누군데, 얘네, ?]"
3,3,"쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...",쑈 하지마라 자식아 음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 기어...,쑈 하지마라 자식 아 음주 1 번은 실수 2 번은 고의 3 번은 인간 쓰레기 다 슬...,"[쑈, 하, 지, 마, 라, 자식, 아, 음주, 1, 번, 은, 실수, 2, 번, ...",쑈 하 지 마 라 자식 아 음주 1 번 은 실수 2 번 은 고의 3 번 은 인간 쓰...,쑈 하지마라 자식 아 음주 1 번은 실수 2 번은 고의 3 번은 인간 쓰레기 다 슬...,"[쑈, 하지마라, 자식, ##아, 음주, 1번, ##은, 실수, 2번, ##은, 고..."
4,4,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검 하수 가지 고 있는 분 께 희망 을 주고 싶은건 가요 ? 수술 하면 이렇게 자...,"[안검, 하, 수, 가지, 고, 있, 는, 분, 께, 희망, 을, 주, 고, 싶, ...",안검 하 수 가지 고 있 는 분 께 희망 을 주 고 싶 은 건가요 ? 수술 하 면 ...,안검 하수 가지 고 있는 분 께 희망 을 주고 싶은건 가요 ? 수술 하면 이렇게 자...,"[안, ##검, ##하, ##수, 가지고, 있는, 분, ##께, 희망을, 주고, 싶..."
...,...,...,...,...,...,...,...,...
506,506,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소 당하면 어마어마한 금액 물어줘 야할 껄 ?,"[얜, 그냥, 봐도, 아니, 다, ㅋ, 고소, 당하, 면, 어마어마, 한, 금액, ...",얜 그냥 봐도 아니 다 ㅋ 고소 당하 면 어마어마 한 금액 물 어 줘야 할껄 ?,얜 그냥 봐도 아니다 ㅋ 고소 당하면 어마어마한 금액 물어줘 야할 껄 ?,"[얜, 그냥, 봐도, 아니다, ㅋ, 고소, ##당하면, 어마어마한, 금액, 물어, ..."
507,507,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야징 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야 징 컨셉 이 바뀌니깐 재미 지 넹,"[대박, 게스트, 꼭, 봐, 야징, 컨셉, 이, 바뀌, 니깐, 재미, 지, 넹]",대박 게스트 꼭 봐 야징 컨셉 이 바뀌 니깐 재미 지 넹,대박 게스트 꼭 봐야 징 컨셉 이 바뀌니깐 재미 지 넹,"[대박, 게스트, 꼭, 봐야, ##징, 컨셉, ##이, 바뀌, ##니깐, 재미, #..."
508,508,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,성형으로 다 뜯어고쳐놓고 예쁜척 성형 전 니 얼굴 다 알고있다 순자처럼 된장냄새나게...,성형 으로 다 뜯어 고쳐놓고 예쁜 척 성형 전 니 얼굴 다 알고있다 순자 처럼 된장...,"[성형, 으로, 다, 뜯어고쳐, 놓, 고, 예쁜, 척, 성형, 전, 니, 얼굴, 다...",성형 으로 다 뜯어고쳐 놓 고 예쁜 척 성형 전 니 얼굴 다 알 고 있 다 순자 처...,성형 으로 다 뜯어 고쳐놓고 예쁜 척 성형 전 니 얼굴 다 알고있다 순자 처럼 된장...,"[성형, ##으로, 다, 뜯어, ##고, ##쳐, ##놓고, 예쁜, ##척, 성형,..."
509,509,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기 는 비슷하다만 전혀 다른 전개 던데 무슨 ㅋㅋㄱ 우리나라 사람 들 은 분위기...,"[분위기, 는, 비슷, 하, 다만, 전혀, 다른, 전개, 던데, 무슨, ㅋㅋ, ㄱ,...",분위기 는 비슷 하 다만 전혀 다른 전개 던데 무슨 ㅋㅋ ㄱ 우리나라 사람 들 은 ...,분위기 는 비슷하다만 전혀 다른 전개 던데 무슨 ㅋㅋㄱ 우리나라 사람 들 은 분위기...,"[분위기, ##는, 비슷, ##하다, ##만, 전혀, ##다른, 전개, ##던데, ..."


## 대회 시작 시점, 기본모델로 결과예측 후 Base점수 잡기

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline('text-classification', model='beomi/beep-KcELECTRA-base-bias', device=0)

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

In [ ]:
test_df['label'] = test_df['clean_comment2'].map(lambda x:pipe(x)[0]['label'])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
test_df

,ID,comment,clean_comment,okt_clean,mecab,mecab_clean,clean_comment2,token,label
0,0,둘다 넘 좋다~행복하세요,둘다 넘 좋다 행복하세요,둘다 넘 좋다 행복하세요,"[둘, 다, 넘, 좋, 다, 행복, 하, 세요]",둘 다 넘 좋 다 행복 하 세요,둘다 넘 좋다 행복하세요,"[둘다, 넘, 좋다, 행복, ##하세요]",none
1,1,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만 원 이하 는 현금 결제 만 하라 고 써놓은 집 우리나라 에 엄청 많은데,"[근데, 만, 원, 이, 하, 는, 현금, 결제, 만, 하, 라고, 써, 놓, 은,...",근데 만 원 이 하 는 현금 결제 만 하 라고 써 놓 은 집 우리나라 에 엄청 많 은데,근데 만 원 이하 는 현금 결제 만 하라 고 써놓은 집 우리나라 에 엄청 많은데,"[근데, 만원, ##이하, ##는, 현금, ##결제, ##만, 하라고, 써, ##놓...",none
2,2,누군데 얘네?,누군데 얘네? 누군데 얘네?,누군데 얘 네 ? 누군데 얘 네 ?,"[누군데, 얘, 네, ?, 누군데, 얘, 네, ?]",누군데 얘 네 ? 누군데 얘 네 ?,누군데 얘 네 ? 누군데 얘 네 ?,"[누군데, 얘네, ?, 누군데, 얘네, ?]",none
3,3,"쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...",쑈 하지마라 자식아 음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 기어...,쑈 하지마라 자식 아 음주 1 번은 실수 2 번은 고의 3 번은 인간 쓰레기 다 슬...,"[쑈, 하, 지, 마, 라, 자식, 아, 음주, 1, 번, 은, 실수, 2, 번, ...",쑈 하 지 마 라 자식 아 음주 1 번 은 실수 2 번 은 고의 3 번 은 인간 쓰...,쑈 하지마라 자식 아 음주 1 번은 실수 2 번은 고의 3 번은 인간 쓰레기 다 슬...,"[쑈, 하지마라, 자식, ##아, 음주, 1번, ##은, 실수, 2번, ##은, 고...",none
4,4,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검 하수 가지 고 있는 분 께 희망 을 주고 싶은건 가요 ? 수술 하면 이렇게 자...,"[안검, 하, 수, 가지, 고, 있, 는, 분, 께, 희망, 을, 주, 고, 싶, ...",안검 하 수 가지 고 있 는 분 께 희망 을 주 고 싶 은 건가요 ? 수술 하 면 ...,안검 하수 가지 고 있는 분 께 희망 을 주고 싶은건 가요 ? 수술 하면 이렇게 자...,"[안, ##검, ##하, ##수, 가지고, 있는, 분, ##께, 희망을, 주고, 싶...",others
...,...,...,...,...,...,...,...,...,...
506,506,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소 당하면 어마어마한 금액 물어줘 야할 껄 ?,"[얜, 그냥, 봐도, 아니, 다, ㅋ, 고소, 당하, 면, 어마어마, 한, 금액, ...",얜 그냥 봐도 아니 다 ㅋ 고소 당하 면 어마어마 한 금액 물 어 줘야 할껄 ?,얜 그냥 봐도 아니다 ㅋ 고소 당하면 어마어마한 금액 물어줘 야할 껄 ?,"[얜, 그냥, 봐도, 아니다, ㅋ, 고소, ##당하면, 어마어마한, 금액, 물어, ...",none
507,507,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야징 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야 징 컨셉 이 바뀌니깐 재미 지 넹,"[대박, 게스트, 꼭, 봐, 야징, 컨셉, 이, 바뀌, 니깐, 재미, 지, 넹]",대박 게스트 꼭 봐 야징 컨셉 이 바뀌 니깐 재미 지 넹,대박 게스트 꼭 봐야 징 컨셉 이 바뀌니깐 재미 지 넹,"[대박, 게스트, 꼭, 봐야, ##징, 컨셉, ##이, 바뀌, ##니깐, 재미, #...",none
508,508,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,성형으로 다 뜯어고쳐놓고 예쁜척 성형 전 니 얼굴 다 알고있다 순자처럼 된장냄새나게...,성형 으로 다 뜯어 고쳐놓고 예쁜 척 성형 전 니 얼굴 다 알고있다 순자 처럼 된장...,"[성형, 으로, 다, 뜯어고쳐, 놓, 고, 예쁜, 척, 성형, 전, 니, 얼굴, 다...",성형 으로 다 뜯어고쳐 놓 고 예쁜 척 성형 전 니 얼굴 다 알 고 있 다 순자 처...,성형 으로 다 뜯어 고쳐놓고 예쁜 척 성형 전 니 얼굴 다 알고있다 순자 처럼 된장...,"[성형, ##으로, 다, 뜯어, ##고, ##쳐, ##놓고, 예쁜, ##척, 성형,...",others
509,509,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기 는 비슷하다만 전혀 다른 전개 던데 무슨 ㅋㅋㄱ 우리나라 사람 들 은 분위기...,"[분위기, 는, 비슷, 하, 다만, 전혀, 다른, 전개, 던데, 무슨, ㅋㅋ, ㄱ,...",분위기 는 비슷 하 다만 전혀 다른 전개 던데 무슨 ㅋㅋ ㄱ 우리나라 사람 들 은 ...,분위기 는 비슷하다만 전혀 다른 전개 던데 무슨 ㅋㅋㄱ 우리나라 사람 들 은 분위기...,"[분위기, ##는, 비슷, ##하다, ##만, 전혀, ##다른, 전개, ##던데, ...",none


In [ ]:
# bias
change = {'none':'none', 'gender':'gender', 'others':'others'}

In [ ]:
# hate
change = {'offensive':'hate', 'none':'none', 'hate':'hate'}
test_df.label = test_df.label.map(change)
test_df

,ID,comment,clean_comment,okt,okt_clean,mecab,mecab_clean,clean_comment2,label
0,0,둘다 넘 좋다~행복하세요,둘다 넘 좋다 행복하세요,"[둘다, 넘, 좋다, 행복하세요]",둘다 넘 좋다 행복하세요,"[둘, 다, 넘, 좋, 다, 행복, 하, 세요]",둘 다 넘 좋 다 행복 하 세요,둘다 넘 좋다 행복하세요,none
1,1,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,"[근데, 만, 원, 이하, 는, 현금, 결제, 만, 하라, 고, 써놓은, 집, 우리...",근데 만 원 이하 는 현금 결제 만 하라 고 써놓은 집 우리나라 에 엄청 많은데,"[근데, 만, 원, 이, 하, 는, 현금, 결제, 만, 하, 라고, 써, 놓, 은,...",근데 만 원 이 하 는 현금 결제 만 하 라고 써 놓 은 집 우리나라 에 엄청 많 은데,근데 만 원 이하 는 현금 결제 만 하라 고 써놓은 집 우리나라 에 엄청 많은데,none
2,2,누군데 얘네?,누군데 얘네? 누군데 얘네?,"[누군데, 얘, 네, ?, 누군데, 얘, 네, ?]",누군데 얘 네 ? 누군데 얘 네 ?,"[누군데, 얘, 네, ?, 누군데, 얘, 네, ?]",누군데 얘 네 ? 누군데 얘 네 ?,누군데 얘 네 ? 누군데 얘 네 ?,hate
3,3,"쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...",쑈 하지마라 자식아 음주 1번은 실수 2번은 고의 3번은 인간쓰레기다 슬금슬금 기어...,"[쑈, 하지마라, 자식, 아, 음주, 1, 번은, 실수, 2, 번은, 고의, 3, ...",쑈 하지마라 자식 아 음주 1 번은 실수 2 번은 고의 3 번은 인간 쓰레기 다 슬...,"[쑈, 하, 지, 마, 라, 자식, 아, 음주, 1, 번, 은, 실수, 2, 번, ...",쑈 하 지 마 라 자식 아 음주 1 번 은 실수 2 번 은 고의 3 번 은 인간 쓰...,쑈 하지마라 자식 아 음주 1 번은 실수 2 번은 고의 3 번은 인간 쓰레기 다 슬...,hate
4,4,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,"[안검, 하수, 가지, 고, 있는, 분, 께, 희망, 을, 주고, 싶은건, 가요, ...",안검 하수 가지 고 있는 분 께 희망 을 주고 싶은건 가요 ? 수술 하면 이렇게 자...,"[안검, 하, 수, 가지, 고, 있, 는, 분, 께, 희망, 을, 주, 고, 싶, ...",안검 하 수 가지 고 있 는 분 께 희망 을 주 고 싶 은 건가요 ? 수술 하 면 ...,안검 하수 가지 고 있는 분 께 희망 을 주고 싶은건 가요 ? 수술 하면 이렇게 자...,none
...,...,...,...,...,...,...,...,...,...
506,506,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,"[얜, 그냥, 봐도, 아니다, ㅋ, 고소, 당하면, 어마어마한, 금액, 물어줘, 야...",얜 그냥 봐도 아니다 ㅋ 고소 당하면 어마어마한 금액 물어줘 야할 껄 ?,"[얜, 그냥, 봐도, 아니, 다, ㅋ, 고소, 당하, 면, 어마어마, 한, 금액, ...",얜 그냥 봐도 아니 다 ㅋ 고소 당하 면 어마어마 한 금액 물 어 줘야 할껄 ?,얜 그냥 봐도 아니다 ㅋ 고소 당하면 어마어마한 금액 물어줘 야할 껄 ?,hate
507,507,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,대박 게스트 꼭 봐야징 컨셉이 바뀌니깐 재미지넹,"[대박, 게스트, 꼭, 봐야, 징, 컨셉, 이, 바뀌니깐, 재미, 지, 넹]",대박 게스트 꼭 봐야 징 컨셉 이 바뀌니깐 재미 지 넹,"[대박, 게스트, 꼭, 봐, 야징, 컨셉, 이, 바뀌, 니깐, 재미, 지, 넹]",대박 게스트 꼭 봐 야징 컨셉 이 바뀌 니깐 재미 지 넹,대박 게스트 꼭 봐야 징 컨셉 이 바뀌니깐 재미 지 넹,none
508,508,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,성형으로 다 뜯어고쳐놓고 예쁜척 성형 전 니 얼굴 다 알고있다 순자처럼 된장냄새나게...,"[성형, 으로, 다, 뜯어, 고쳐놓고, 예쁜, 척, 성형, 전, 니, 얼굴, 다, ...",성형 으로 다 뜯어 고쳐놓고 예쁜 척 성형 전 니 얼굴 다 알고있다 순자 처럼 된장...,"[성형, 으로, 다, 뜯어고쳐, 놓, 고, 예쁜, 척, 성형, 전, 니, 얼굴, 다...",성형 으로 다 뜯어고쳐 놓 고 예쁜 척 성형 전 니 얼굴 다 알 고 있 다 순자 처...,성형 으로 다 뜯어 고쳐놓고 예쁜 척 성형 전 니 얼굴 다 알고있다 순자 처럼 된장...,hate
509,509,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,"[분위기, 는, 비슷하다만, 전혀, 다른, 전개, 던데, 무슨, ㅋㅋㄱ, 우리나라,...",분위기 는 비슷하다만 전혀 다른 전개 던데 무슨 ㅋㅋㄱ 우리나라 사람 들 은 분위기...,"[분위기, 는, 비슷, 하, 다만, 전혀, 다른, 전개, 던데, 무슨, ㅋㅋ, ㄱ,...",분위기 는 비슷 하 다만 전혀 다른 전개 던데 무슨 ㅋㅋ ㄱ 우리나라 사람 들 은 ...,분위기 는 비슷하다만 전혀 다른 전개 던데 무슨 ㅋㅋㄱ 우리나라 사람 들 은 분위기...,hate


In [ ]:
# hate_list = test_df['label'].tolist()   # hate
bias_list = test_df['label'].tolist()   # bias

team_best = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/submission/rsf_submission.csv')
team_best

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,none,hate
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,others,hate
509,509,others,hate


In [ ]:
# team_best['hate'] = hate_list   # hate
team_best['bias'] = bias_list   # bias

In [ ]:
team_best

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,others,hate
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,others,hate
509,509,none,hate


In [ ]:
team_best.to_csv('/content/drive/MyDrive/AI_team/competition3/submission/bias_test_test.csv', index=False)

In [ ]:
# Train 데이터 bias 예측 점수 확인
train_df['label'] = train_df['comment'].map(lambda x:pipe(x)[0]['label'])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
train_df.shape[0]

8367

In [ ]:
train_df[train_df['bias']==train_df['label']]   # train에서의 정확도 98.7809 ( 8265/8367 )

,title,comment,bias,hate,label
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,none
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,none
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,others
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,none
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,none
...,...,...,...,...,...
8362,"""배우 이필립, SNS 스타 연인에게 초호화 프러포즈 눈길""",아니 근데.튜닝한사람은 프러포즈받지도.결혼도못함?ㅋㅋㅋ지들은 돈없어서 못하는것들이ㅋ...,others,hate,others
8363,"""[SC이슈]""""마약·백스텝·김새롬 탓"""" '실형 피한' 이찬오, 이미지는 치명상(...",그러니깐 여자를 잘만나야되~징글징글한것들 만나면 인생 끝가지 돌아가게 되는듯.. 근...,gender,hate,gender
8364,"""[POP이슈]""""그들만의 세상""""…홍상수♥김민희, 새해데이트에 '반응싸늘'""",참으로 아름다운 커플입니다. 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다 ...,none,none,none
8365,[종합] '시크릿 마더' 김소연 누가 죽였나…송윤아와 갈등,재미가 없어요,none,none,none


In [ ]:
from sklearn.metrics import f1_score

y_true = list(train_df['bias'])
y_pred = list(train_df['label'])
f1_score(y_true, y_pred, average='macro')

0.983661525444742

In [ ]:
# Train data 중 Bias 평가에서 틀린 것 
train_df[train_df['bias']!=train_df['label']] 

,title,comment,bias,hate,label
144,"""""""남편 고등학생 때 임신""""…홍영기, '동치미' 출연 예고만으로 화제 [종합]""",제정신아니네 고딩과과계한것도웃기지만 성인이피임도안하나 애는 남자한테줘버리고결혼은하지...,gender,hate,others
191,"""[SC이슈]'송혜교♥' 송중기 """"사랑한다, 살아간다"""" 찬란한 소년美""","중기야,,, 왜 히필 늙다리한테 물려서,,, ㅠㅠ",gender,hate,others
216,"""“죄책감·부끄럽다”..판빙빙, 탈세 4개월만 사과와 반성 ‘논란 일단락’(종합)[...",ㅈㄹ 하고 자빠졋네..판빙빙 얼굴바 뭐 하냐...탈세 자랑스러운 일이냐?,none,hate,others
330,"""""""인형 닮아가네""""…이하이, 볼살 통통 근황 '깜짝'""",상품성 제로. 진자 대형기획사빨아녓슴 애진장에 끝낫지..,others,hate,none
367,"""한혜진, 전현무와 결별 후 밝은 모습 """"인생술집, 고마워요""""""",몸매는 섹시,others,hate,gender
...,...,...,...,...,...
8034,"""[夜TV] """"잘 산것 같아""""…'너는내운명' 노사연♥이무송, 더 감동인 이유""",두 분 권태기없이 이뿌게 사랑하머 아껴주는 모습 보기 좋아요~~사연언니 천상 여자여...,gender,none,none
8043,'신소율 남자친구' 뮤지컬배우 김지철은 누구?,그냥 헬쥐팀 처럼 생겼네 이년도 곳 저넘한테 차이고 19년 3월 30일날 혼나 와서...,others,hate,gender
8160,"""'전원일기 배우→무용수'…'불타는 청춘' 조하나, 추억 속 여배우[SS리뷰]""",30대중후반대같네 캐동안,others,none,none
8176,"""'톱스타 유백이' 김지석, 전소민에 """"네가 내 프레임 안에 들어오는거야""""[★밤...",미쳤다고 톱스타가 저런 시골촌구석에 가서 저런 촌년이랑 사랑에 빠지나 우리나라 드라...,others,hate,gender


In [ ]:
# Test 데이터 bias 예측 하기
test_df['label'] = test_df['comment'].map(lambda x:pipe(x)[0]['label'])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
test_df.head()

,ID,title,comment,label,label2
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요,none,none
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,none,none
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?,none,hate
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...",none,hate
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,others,hate


In [ ]:
bias_list = test_df['label'].tolist()
print(bias_list)
print(len(bias_list))

['none', 'none', 'none', 'none', 'others', 'none', 'others', 'none', 'gender', 'none', 'gender', 'gender', 'none', 'others', 'none', 'gender', 'none', 'gender', 'none', 'none', 'gender', 'none', 'others', 'none', 'none', 'none', 'gender', 'none', 'none', 'none', 'others', 'gender', 'none', 'others', 'none', 'gender', 'none', 'none', 'none', 'none', 'others', 'none', 'others', 'none', 'none', 'none', 'none', 'none', 'none', 'gender', 'none', 'none', 'none', 'gender', 'none', 'gender', 'gender', 'none', 'none', 'others', 'none', 'gender', 'none', 'gender', 'none', 'none', 'none', 'none', 'none', 'gender', 'none', 'none', 'none', 'none', 'gender', 'none', 'gender', 'others', 'gender', 'none', 'none', 'none', 'others', 'none', 'none', 'none', 'none', 'others', 'gender', 'none', 'none', 'none', 'none', 'none', 'none', 'gender', 'none', 'others', 'none', 'none', 'none', 'none', 'none', 'none', 'gender', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'none', 'others', 'others

In [ ]:
submit = pd.read_csv(os.path.join(args.data_dir,'sample_submission.csv'))
submit['bias'] = bias_list
submit

In [ ]:
pipe2 = pipeline('text-classification', model='beomi/beep-KcELECTRA-base-hate', device=0)

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/544 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/769k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Train data로 hate 평가
train_df['label2'] = train_df['comment'].map(lambda x:pipe2(x)[0]['label'])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
# offensive를 hate로 바꿔주기
change = {'offensive':'hate', 'none':'none', 'hate':'hate'}
train_df.label2 = train_df.label2.map(change)

In [ ]:
# Train data 중 Hate 평가에서 틀린 것   # 정확도  96.3069 ( 8058/8367 )
train_df[train_df['hate']!=train_df['label2']] 

,title,comment,bias,hate,label,label2
15,"""[종합] """"부모님 스트레스多""""…'편스토랑' 이혜성, ♥전현무와 공개 열애 고충...",서울대 이혜성 >>>>>> 넘사 >>>>>>> 동덕여대 한혜진. 전현무도 결혼할 나...,gender,hate,gender,none
22,"""[룩@차이나] 中관계자 """"판빙빙, 조사받고 귀가한 지 한참 됐다""""""",무슨상관이냐고 기자양반ㅠ,none,none,none,hate
67,"""[Oh!커피 한 잔①] '신과함께2' 이정재 """"염라언니, 내가 이런 역할까지 하...",니 아부지가 염라언니였어 ㅍㅎㅎ,none,none,none,hate
82,"""[단독] 이민호 측근 """"수지와 재결합 NO, 다시 친구로 지내는중""""""",수지는 참 애가 보는 눈이 참 없네 ㅋㅋ 뭐가 아쉽다고 이민호 같은 애랑 ㅋㅋ 에휴...,none,none,none,hate
87,"""오윤아 “아들, 발달장애…죄스러운 마음으로 살고 있다” [인터뷰]""",이 누나도 늙네...,others,none,others,hate
...,...,...,...,...,...,...
8178,"""[★밤TView]'불후' 멜로망스, 감미로운 목소리..첫 출연에 최종 勝""",멜로망스 씹 거품 그 자체,none,hate,none,none
8226,"""SBS """"고현정 교체 검토중""""…수목극 '리턴' 파행(종합)""",자살은 하지마시길...,none,hate,none,none
8249,"""'율희♥' 최민환, 생후 11일된 '짱이' 사진과 산모 율희 사진 공개""",역시 어린엄마가 짱이야 여자는 어려서 결혼해야해,gender,hate,gender,none
8292,"""""""벗방·몸캠 한적 없어""""…브레이브걸스 출신 BJ서아, 각종 의혹 해명""",여기 댓글에 자격지심 느끼는 애들 화사닮음ㅋ,others,hate,others,none


In [ ]:
y_true2 = list(train_df['hate'])
y_pred2 = list(train_df['label2'])
f1_score(y_true2, y_pred2, average='macro')

0.9625274874561656

In [ ]:
y_pred2_rvs = list()

for label in y_pred2:
    if label == 'hate':
        y_pred2_rvs.append('none')
    else:
        y_pred2_rvs.append('hate')

len(y_pred2_rvs)

8367

In [ ]:
# 답 rvs 한 f1 score
f1_score(y_true2, y_pred2_rvs, average='macro')

0.036865298910052885

In [ ]:
0.9625274874561656 + 0.036865298910052885

0.9993927863662184

In [ ]:
test_df['label2'] = test_df['comment'].map(lambda x:pipe2(x)[0]['label'])
test_df

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


,ID,title,comment,label,label2
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요,none,none
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,none,none
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?,none,offensive
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...",none,hate
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,others,offensive
...,...,...,...,...,...
506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로… 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,none,none
507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명…""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,none,none
508,508,"[SS인터뷰①]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,others,hate
509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인→여전히 '핫'(종합)",분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,others,offensive


In [ ]:
change = {'offensive':'hate', 'none':'none', 'hate':'hate'}
test_df.label2 = test_df.label2.map(change)
test_df

,ID,title,comment,label,label2
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요,none,none
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,none,none
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?,none,hate
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ...",none,hate
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...,others,hate
...,...,...,...,...,...
506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로… 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어마어마한 금액 물어줘야할껄?,none,none
507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명…""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,none,none
508,508,"[SS인터뷰①]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,others,hate
509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인→여전히 '핫'(종합)",분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,others,hate


In [ ]:
hate_list = test_df['label2'].tolist()
print(hate_list)
print(len(hate_list))

['none', 'none', 'hate', 'hate', 'hate', 'hate', 'hate', 'hate', 'hate', 'none', 'none', 'hate', 'none', 'hate', 'none', 'hate', 'hate', 'hate', 'none', 'none', 'hate', 'hate', 'hate', 'hate', 'hate', 'none', 'hate', 'none', 'none', 'none', 'hate', 'hate', 'hate', 'hate', 'none', 'hate', 'none', 'none', 'hate', 'none', 'hate', 'hate', 'hate', 'none', 'none', 'hate', 'hate', 'none', 'none', 'hate', 'hate', 'none', 'hate', 'hate', 'hate', 'hate', 'hate', 'none', 'none', 'hate', 'hate', 'none', 'none', 'hate', 'hate', 'hate', 'hate', 'hate', 'none', 'hate', 'none', 'hate', 'none', 'hate', 'hate', 'none', 'hate', 'hate', 'hate', 'hate', 'hate', 'hate', 'hate', 'none', 'hate', 'none', 'none', 'hate', 'hate', 'none', 'none', 'hate', 'hate', 'none', 'hate', 'hate', 'hate', 'hate', 'hate', 'hate', 'none', 'hate', 'none', 'none', 'hate', 'none', 'hate', 'hate', 'hate', 'none', 'none', 'hate', 'hate', 'hate', 'hate', 'hate', 'hate', 'none', 'none', 'none', 'none', 'none', 'none', 'hate', 'hate',

In [ ]:
submit['hate'] = hate_list

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,others,hate
...,...,...,...
506,506,none,none
507,507,none,none
508,508,others,hate
509,509,others,hate


In [ ]:
submit.to_csv('/content/drive/MyDrive/AI_team/competition3/submission/base_submission.csv')

## 직접 분류하기

In [ ]:
# 점수 하락
noneToOthers = [16,18,54,81,107,118,131,136,150,193,213,224,260,274,343,406,439,461,462,492,500]
noneToGender = [333,390,476]

In [ ]:
# team best score submit
team_best_submit = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/submission/team_best_score_7767.csv')

In [ ]:
# 점수 하락
noneToOthers = [16,23,45,59,67,92,117,125,134,136,213,224,322,379,413,417,421,439,452,477]

In [ ]:
# 점수 조금 하락
noneToOthers = [268,304,319,321,329,379,406,439]
noneToGender = [451]

In [ ]:
# 점수 조금 하락
noneToOthers = [304,319,329,379,406,421,439]
noneToGender = [390, 451]

In [ ]:
team_best_submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,none,hate
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,others,hate
509,509,others,hate


In [ ]:
for i in noneToOthers:
    team_best_submit['bias'].iloc[i] = 'others'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
for i in noneToGender:
    team_best_submit['bias'].iloc[i] = 'gender'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
team_best_submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,none,hate
4,4,none,hate
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,others,hate
509,509,others,hate


In [ ]:
team_best_submit.to_csv('/content/drive/MyDrive/AI_team/competition3/submission/tokenchange_bias5.csv', index=False)

## 전처리한 Test data 파일 확인하기

In [ ]:
test_df2 = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/data/test_spacing.csv')
test_df3 = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/data/test_clear.csv')
test_df4 = pd.read_csv('/content/drive/MyDrive/AI_team/competition3/data/test_both.csv')

In [ ]:
test_df2['label'] = test_df2['comment'].map(lambda x:pipe(x)[0]['label'])
test_df2['label2'] = test_df2['comment'].map(lambda x:pipe2(x)[0]['label'])

change = {'offensive':'hate', 'none':'none', 'hate':'hate'}
test_df2.label2 = test_df2.label2.map(change)

bias_list2 = test_df2['label'].tolist()
hate_list2 = test_df2['label2'].tolist()


submit2 = pd.read_csv(os.path.join(args.data_dir,'sample_submission.csv'))
submit2

In [ ]:
submit2['bias'] = bias_list2
submit2['hate'] = hate_list2

In [ ]:
submit2

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,gender,hate
4,4,others,hate
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,others,hate
509,509,none,hate


In [ ]:
len(submit2[submit2['bias']!=submit['bias']])

27

In [ ]:
submit2.to_csv('/content/drive/MyDrive/AI_team/competition3/submission/base_submission_spacing.csv')

In [ ]:
test_df3['label'] = test_df3['comment'].map(lambda x:pipe(x)[0]['label'])
test_df3['label2'] = test_df3['comment'].map(lambda x:pipe2(x)[0]['label'])

change = {'offensive':'hate', 'none':'none', 'hate':'hate'}
test_df3.label2 = test_df3.label2.map(change)

bias_list3 = test_df3['label'].tolist()
hate_list3 = test_df3['label2'].tolist()


submit3 = pd.read_csv(os.path.join(args.data_dir,'sample_submission.csv'))
submit3

submit3['bias'] = bias_list2
submit3['hate'] = hate_list2

submit3

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:978: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,hate
3,3,gender,hate
4,4,others,hate
...,...,...,...
506,506,none,hate
507,507,none,none
508,508,others,hate
509,509,none,hate


In [ ]:
len(submit3[submit3['hate']!=submit2['hate']])

0

In [ ]:
submit3.to_csv('/content/drive/MyDrive/AI_team/competition3/submission/base_submission_clear.csv')

In [ ]:
# test 원본, test spacing 비교
print(len(test_df[test_df['comment']!=test_df2['comment']]))

# test 원본, test clear 비교
print(len(test_df[test_df['comment']!=test_df3['comment']]))

# test space, test clear 비교
print(len(test_df2[test_df2['comment']!=test_df3['comment']]))

418
423
31


In [ ]:
test_df3

,Unnamed: 0,Unnamed: 0.1,ID,title,comment,label,label2
0,0,0,0,"류현경 박성훈, 공개연애 4년차 애정전선 이상 ..""의지 많이 된다""[종합]",둘 다 넘 좋다~행복하세요,none,none
1,1,1,1,"""현금 유도+1인 1라면?"" '골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만 원 이하는 현금 결제만 하라고 써놓은 집 우리나라에 엄청 많은데,none,none
2,2,2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?,none,hate
3,3,3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개 ""결혼,출산 숨겼다""","쑈 하지 마라 짜식아! 음주 1번은 실수, 2번은 고의, 3번은 인간 쓰레기다.슬금...",gender,hate
4,4,4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은 건가요? 수술하면 이렇게 자연스러워진다...,others,hate
...,...,...,...,...,...,...,...
506,506,506,506,"[N이슈] 최율, 조재현 성추행 의혹 폭로 소속사 ""상황 파악 중""",얜 그냥 봐도 아니다 ㅋ 고소당하면 어 마어마한 금액 물어줘야 할 껄?,none,hate
507,507,507,507,"해투4' 이서진, 한지민 '대본 리딩 격리설' 해명 ""날씨가 좋아서"" [SC컷]",대박 게스트... 꼭 봐야 징~ 컨셉이 바뀌니깐 재미지넹,none,none
508,508,508,508,"[SS인터뷰 ]박민영 ""'김비서' 행복했다..열애설엔 당당..미소였으니까""",성형으로 다 뜯어고쳐 놓고 예쁜 척. 성형 전 니 얼굴 다 알고 있다. 순자처럼 된...,others,hate
509,509,509,509,"[POP이슈]""사실무근"" 'SKY캐슬' 측 '위올라이' 표절설 부인 여전히 '핫'(종합)",분위기는 비슷하다 만 전혀 다른 전개 던데 무슨 ㅋㅋㄱ 우리나라 사람들은 분위기만 ...,none,hate


In [ ]:
pd.crosstab(test_df3.label, test_df3.label2, margins=True)

label2,hate,none,All
label,,,
gender,98,5,103
none,174,174,348
others,58,2,60
All,330,181,511


In [ ]:
test_df3[(test_df3['label2']=='none') & (test_df3['label']!='none')]

,Unnamed: 0,Unnamed: 0.1,ID,title,comment,label,label2
61,61,61,61,"""응암동의 최수종"" 이용진, 7년 열애 비연예인 여친과 오늘(14일) 결혼(종합)[...",왜 이제서야 확 뜨 신 겁니꼬. 넘웃기고.여자들한테 인기 많을 스타일인데 7년이나 ...,gender,none
215,215,215,215,"송혜교, 인형이 나타났다 긴머리로 리즈 경신",이쁘긴 겁나이쁜데... 이 제 이 혼녀ㅠ,gender,none
263,263,263,263,"세젤예' 김해숙, 폐암 말기 3개월 시한부 말도 못하고 오열+갈등 [종합]",상상암이랍니다 글 내려주세요,others,none
333,333,333,333,"[인터뷰 ] 수애 ""노출·베드신 장르, 부담보다 도전이라 생각했다""",도끼는 보여야 파격 노출이지,gender,none
420,420,420,420,"[단독] 배우 박하나, 한의사와 열애 중",한의사는 합법적 무당이라 ...,others,none
483,483,483,483,"팬심'마저 등 돌린, 이태임의 '자충수'",아이가 불쌍해... 아닌가.. 아이가 남자면 엄마 목욕하는 거 보고 므흣~~~~,gender,none
501,501,501,501,"[Oh!쎈 이슈] ""하차·교체 재촬영""..윤태영 '음주운전'에 애꿎은 '백일'만 피...",돈이 넘쳐나는 사람이 대리비가 아까웠냐..,gender,none


In [ ]:
a = pd.crosstab(train_df.bias, train_df.hate, margins=True)

In [ ]:
train_df[(train_df['hate']=='none') & (train_df['bias']!='none')]

,title,comment,bias,hate,label
43,"""[SC초점] '연락두절' 오달수·'미국行' 조근현, 성추문 언제쯤 입 열까?""",오달수 조재현은 인기 많으니까 좀 봐주자~ 달수형님 안나오는 영화 재미없다~,others,none,others
44,"""'미우새' 배기성 12세 연하 아내, 세계 음식 대접..금손 자랑""",여자는 얼굴보다 요리실력. 근데 얼굴도 이쁘심.,gender,none,gender
54,"""폭행→상처→동영상..구하라, 진흙탕 싸움 조짐[종합]""",남자가 얼마나 속상했으면 저럴까... 같은 잘생기고 멋진 남자로서 이해한다,gender,none,gender
87,"""오윤아 “아들, 발달장애…죄스러운 마음으로 살고 있다” [인터뷰]""",이 누나도 늙네...,others,none,others
120,"""[단독] B.A.P 출신 대현 “해체 후 무너졌던 시간, 한 단계 성장했죠”(인터뷰)""",군대나 가세요,others,none,others
...,...,...,...,...,...
8263,[DA:이슈] ‘구하라 비보’ 최종범 항소심에 영향?…법조계 “‘공소권 없음’ 아냐”,최근 본 사회면 기사중에 유.일.하.게. 여성 댓글 비율이 높네,gender,none,gender
8270,"""'보니하니' 측 """"당당맨 최영수, 채연에 폭력 NO…심한 장난으로 이어진 것""""...",나라가 점점 PC화 되간다 망조가 들엇누,others,none,others
8337,"""고준희→정은채→한효주, 버닝썬 논란 희생양 된 여배우들[이슈와치]""",여자 연예인들은 참 힘들겠다ㅠㅠ,gender,none,gender
8353,"""성폭행 막은 한상규 """"나도 놀라..큰 피해 없어 다행""""(직격 인터뷰)""",다 좋은데 남양유업 같은곳 위해서 일하지마라,others,none,others
